In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from unidecode import unidecode

/Users/abemankavil/Desktop/Python/Projects/kNN_NBA_Classifier/kNN_NBA_env/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Use requests pkg to load in raw bytes of HTML

### .decode() changes bytes to str

In [2]:
resp = requests.get('https://www.basketball-reference.com/leagues/NBA_2023_per_game.html')
html_data = resp.content.decode()

soup = BeautifulSoup(html_data)

### find_all() selects HTML lines based on tags

In [3]:

# [1:] in next line to remove 'Rk' attribute that is arbitrary depending on sorting rule
attribute_headers = [header.getText() for header in soup.find_all('tr',limit=1)[0].find_all('th')][1:]

rows = soup.find_all('tbody',limit=1)[0].find_all('tr',class_='full_table')
player_stats = np.array([[attr.getText() for attr in row.find_all('td') ] for row in rows])


In [4]:

df_NBA_Player_Stats = pd.DataFrame(player_stats, columns=attribute_headers)
df_NBA_Player_Stats = df_NBA_Player_Stats.apply(pd.to_numeric, errors='ignore')

df_NBA_Player_Stats['Player'] = df_NBA_Player_Stats['Player'].apply(unidecode)

df_NBA_Player_Stats.head()


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,23,TOR,44,11,22.0,3.6,7.5,0.476,...,0.691,1.9,4.3,6.1,1.0,0.6,0.6,1.2,2.0,9.4
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,Bam Adebayo,C,25,MIA,64,64,35.1,8.3,15.4,0.541,...,0.807,2.4,7.0,9.4,3.3,1.2,0.8,2.6,2.9,21.0
3,Ochai Agbaji,SG,22,UTA,45,8,17.3,2.0,4.6,0.440,...,0.742,0.6,1.1,1.8,0.7,0.2,0.2,0.4,1.5,5.7
4,Santi Aldama,PF,22,MEM,63,18,22.1,3.3,7.0,0.475,...,0.719,1.0,3.6,4.6,1.2,0.6,0.7,0.7,1.9,9.3


In [5]:
#For salary data
resp = requests.get('https://hoopshype.com/salaries/players/')
html_data = resp.content.decode()

soup = BeautifulSoup(html_data)

In [6]:
attribute_headers = [header.getText().strip() for header in soup.find_all('tr',limit=1)[0].find_all('td', limit=3)][1:]
attribute_headers[1] += ' Salary'

In [7]:

rows = soup.find_all('tbody',limit=1)[0].find_all('tr')
rows[0].find_all('td')[1:3][1].getText().strip('')
player_salaries = np.array([[attr.getText().strip('$\t\n').replace(',','') for attr in row.find_all('td')[1:3] ] for row in rows])

In [8]:
df_salaries = pd.DataFrame(player_salaries, columns=attribute_headers)
df_salaries['2022/23 Salary'] = df_salaries['2022/23 Salary'].apply(int)
df_salaries['Player'] = df_salaries['Player'].apply(unidecode)


#### Next step is to use fuzzy wuzzy to connect nicknames of players that didn't match during merge above ('https://towardsdatascience.com/fuzzywuzzy-basica-and-merging-datasets-on-names-with-different-transcriptions-e2bb6e179fbf')

### Use FuzzyWuzzy method process.extractOne() to merge along names including nicknames and truncated names from different sets

In [9]:
names_extract = df_NBA_Player_Stats['Player'].apply(process.extractOne, args=(df_salaries['Player'],))
for i, name in df_NBA_Player_Stats['Player'].items():
        if names_extract[i][1] >= 86:
            df_NBA_Player_Stats.at[i,'Player'] = names_extract[i][0]


In [10]:
df_NBA_Player_Stats


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,23,TOR,44,11,22.0,3.6,7.5,0.476,...,0.691,1.9,4.3,6.1,1.0,0.6,0.6,1.2,2.0,9.4
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,Bam Adebayo,C,25,MIA,64,64,35.1,8.3,15.4,0.541,...,0.807,2.4,7.0,9.4,3.3,1.2,0.8,2.6,2.9,21.0
3,Ochai Agbaji,SG,22,UTA,45,8,17.3,2.0,4.6,0.440,...,0.742,0.6,1.1,1.8,0.7,0.2,0.2,0.4,1.5,5.7
4,Santiago Aldama,PF,22,MEM,63,18,22.1,3.3,7.0,0.475,...,0.719,1.0,3.6,4.6,1.2,0.6,0.7,0.7,1.9,9.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,0.545,...,0.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4
513,Trae Young,PG,24,ATL,61,61,35.0,8.5,19.6,0.433,...,0.889,0.8,2.2,3.0,10.1,1.1,0.1,4.1,1.5,26.8
514,Omer Yurtseven,C,24,MIA,1,0,7.0,0.0,1.0,0.000,...,NaN,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
515,Cody Zeller,C,30,MIA,10,0,13.0,1.5,2.9,0.517,...,0.739,1.6,1.9,3.5,0.6,0.2,0.4,0.8,2.3,4.7


### Save Df to .csv file locally

In [11]:
df_NBA_complete = pd.merge(df_NBA_Player_Stats,df_salaries, on='Player')

df_NBA_complete.to_csv('NBA_Player_Stats.csv',index=False)